In [1]:
import numpy as np

In [119]:
def importancia(W: np.array) -> np.array:
    I = np.zeros(W.shape[1])  # Vetor de importância
    for j in range(W.shape[1]):
        soma = np.sum(np.abs(W[:, j]))
        I[j] = np.sqrt(soma)
    return I

def c(V: list, W: np.array, I: np.array) -> float:
    c_value = 0  # Valor de c
    for j in range(W.shape[1]):
        if any(W[i, j] > 0 for i in V):  # Se a característica j é relevante
            c_value += I[j]
    return c_value

def guloso(W: np.array, I: np.array, B: int) -> list:
    nao_selecionados = list(range(W.shape[0]))  # Lista de índices não selecionados
    V = []  # Conjunto de características selecionadas
    itens = 0  # Número de elementos selecionados
    c_value = 0  # Valor de c
    
    while itens < B and nao_selecionados:
        best_gain = -np.inf  # Valor de ganho máximo
        best_item = None  # Índice do melhor item 
        
        for item in nao_selecionados:  # Itera sobre os itens não selecionados
            lista_temp = V + [item]
            gain = c(lista_temp, W, I) - c(V, W, I)
            if gain > best_gain:
                best_gain = gain  # Atualiza o valor de ganho máximo
                best_item = item  # Atualiza o melhor item
        
        if best_item is not None:
            V.append(best_item)  # Adiciona o melhor item ao conjunto
            nao_selecionados.remove(best_item)  # Remove o melhor item dos não selecionados
            itens += 1
            c_value += best_gain
    
    return V

In [120]:
# Exemplo de uso
W = np.array([[1, 2, 1], [-1, 0, 2], [-2, 2, -2]])  # Exemplo de matriz W
I = importancia(W)
print("Matriz de importância: ", I)

V = [0, 1]  # Lista de índices das instâncias selecionadas
print("Cobertura: ", c(V, W, I))

V_linha = guloso(W, I, 2)
print("Melhor conjunto: ", V_linha)

Matriz de importância:  [2.         2.         2.23606798]
Cobertura:  6.23606797749979
Melhor conjunto:  [0, 1]
